In [71]:
#Reading files
import duckdb

#Reviews-----------
db = duckdb.read_csv(r"C:\Users\Cash\Downloads\user_reviews.csv\user_reviews.csv")
db.write_parquet(r"out.parquet")
# Critic names
db = duckdb.read_csv(r"C:\Users\Cash\Downloads\critic_reviews.csv\critic_reviews.csv")
db.write_parquet(r"out1.parquet")
# Movie names
db = duckdb.read_csv(r"C:\Users\Cash\Downloads\movies.csv\movies.csv")
db.write_parquet(r"out2.parquet")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [60]:
# Number of diferent movieIds
from duckdb.experimental.spark.sql import SparkSession as session
from duckdb.experimental.spark.sql.functions import lit, col

spark = session.builder.getOrCreate()

db_spark = spark.read.parquet(r"out.parquet")


db_spark.groupBy("movieId").count().show()



Hola mundo
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        11473 │
└──────────────┘



In [31]:
# Getting the columns names
import time
# start = time.perf_counter()
con = duckdb.connect()
dl = con.query("""
SELECT *
FROM db
LIMIT 1;
""")
columns_list = dl.to_df().columns.to_list()
for i in columns_list : print(i)

movieId
rating
quote
reviewId
isVerified
isSuperReviewer
hasSpoilers
hasProfanity
score
creationDate
userDisplayName
userRealm
userId


In [94]:
Using PySpark to create more tables
spark = session.builder.getOrCreate()

db_spark1 = spark.read.parquet(r"out1.parquet")
db_spark2 = spark.read.parquet(r"out2.parquet")

# Reviews
db_spark = spark.read.parquet(r"out.parquet")
db_spark.createOrReplaceTempView("db_spark")
# Critics
db_spark1.createOrReplaceTempView("db_spark1")
# Movie name
db_spark2.createOrReplaceTempView("db_spark2")

In [166]:
# Name of the top 5 most reviewed movies
spark.sql("""
SELECT db_spark2.movieTitle, COUNT(*) as Total_reviews
FROM db_spark2
INNER JOIN db_spark
ON db_spark2.movieId = db_spark.movieId
GROUP BY db_spark2.movieTitle
ORDER BY Total_reviews desc
LIMIT 5;
""")

┌───────────────────────────────────────────────┬───────────────┐
│                  movieTitle                   │ Total_reviews │
│                    varchar                    │     int64     │
├───────────────────────────────────────────────┼───────────────┤
│ Titanic                                       │       1157597 │
│ The Lord of the Rings: The Return of the King │        730791 │
│ Harry Potter and the Goblet of Fire           │        694063 │
│ Shrek 2                                       │        681042 │
│ Charlie and the Chocolate Factory             │        661197 │
└───────────────────────────────────────────────┴───────────────┘

In [164]:
# Number of reviews created the last year
reviews_frame = spark.sql("""
SELECT db_spark2.movieTitle, COUNT(*) as Total_reviews
FROM db_spark2
INNER JOIN db_spark
ON db_spark2.movieId = db_spark.movieId
WHERE db_spark.creationDate = (
SELECT MAX(db_spark.creationDate)
FROM db_spark
)
GROUP BY db_spark2.movieTitle
ORDER BY Total_reviews desc;
""")

reviews_frame

┌────────────────────────────────────────────────────┬───────────────┐
│                     movieTitle                     │ Total_reviews │
│                      varchar                       │     int64     │
├────────────────────────────────────────────────────┼───────────────┤
│ Divorce in the Black                               │            24 │
│ Descendants: The Rise of Red                       │             8 │
│ The Garfield Movie                                 │             2 │
│ Kingdom of the Planet of the Apes                  │             2 │
│ Justice League: Crisis on Infinite Earths Part One │             1 │
│ Hannah Berner: We Ride at Dawn                     │             1 │
│ The Exorcism                                       │             1 │
│ Dragonkeeper                                       │             1 │
│ Indian                                             │             1 │
└────────────────────────────────────────────────────┴───────────────┘

In [170]:
# Number of reviews that have spoilers
spark.sql("""
SELECT COUNT(*) as Total_Spoilers
FROM db_spark
WHERE hasSpoilers = true
;
""")


┌────────────────┐
│ Total_Spoilers │
│     int64      │
├────────────────┤
│          11473 │
└────────────────┘

In [172]:
# Number of reviews that the score is higher than 4

spark.sql("""
SELECT COUNT(*) as Reviews_more_than_4
FROM db_spark
WHERE score > 4;
""")


┌─────────────────────┐
│ Reviews_more_than_4 │
│        int64        │
├─────────────────────┤
│            21012715 │
└─────────────────────┘

In [185]:
# Name of movies whose score is higher or equal to 4
spark.sql(
"""
SELECT db_spark2.movieTitle,db_spark.score as Final_score
FROM db_spark2
INNER JOIN db_spark
ON db_spark2.movieId = db_spark.movieId
WHERE db_spark.score >= 4
ORDER BY Final_score desc;
""")
    

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────────────────────────────────────┬─────────────┐
│                  movieTitle                   │ Final_score │
│                    varchar                    │   double    │
├───────────────────────────────────────────────┼─────────────┤
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│ Rocky                                         │         5.0 │
│   ·                                           │          ·  │
│   ·                                   